In [4]:
#Step 0. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다
from bs4 import BeautifulSoup     
from selenium import webdriver
import time

from selenium.webdriver.common.action_chains import ActionChains

# query_txt = input('동네이름?: ')

query_txt= "여의도동"

#Step 1. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "C:/chrome/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get("https://www.yogiyo.co.kr/mobile/?gclid=Cj0KCQjw5JSLBhCxARIsAHgO2SeethOJmtB_u40Cag3kxV42PsPe6iVAL09MEeLNiWcviaLln5OrAyQaAkaDEALw_wcB#/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C/150010/")


In [ ]:
# 점포를 1개 누른이후에 크롤링을 함

In [7]:
from bs4 import BeautifulSoup     
from selenium import webdriver
import time

from selenium.webdriver.common.action_chains import ActionChains
import re
import pandas as pd
# 선택되어지고 나서 누르는셀# 선택되어지고 나서 누르는셀# 선택되어지고 나서 누르는셀
# 선택되어지고 나서 누르는셀

# 한판 돌리는데 시간이 얼마나 걸릴지 한번 측정해봄
import time
stime = time.time()



# 클린 리뷰를 누르는 명령어
element1 = driver.find_element_by_xpath("/html/body/div[6]/div[2]/div[1]/ul/li[2]/a")
ActionChains(driver).move_to_element(element1).click(element1).perform()

# driver.back() #뒤로가기

# 사진 리뷰만 클릭하는 명령어
# time.sleep(1)
# element1 = driver.find_element_by_xpath("/html/body/div[6]/div[2]/div[1]/div[5]/div[2]/div/div/div/label")
# ActionChains(driver).move_to_element(element1).click(element1).perform()
####################################

#k라는 변수를 더보기 누르는셀과 연결됨
k=int(12)
#40번을 아래로 내리는 명령어
for i in range(0,40,1):
    # 다섯번 긁을때마다 표기를 해주도록함
    if i%5 == 0:
        print(i," 번 긁었어요")
    try:        
        time.sleep(0.3) 
        #아래로 내리는 명령어
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")       
        #아래에 더보기를 누르는 것
        kk= "/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{k}]/a".format(k=k)
        element1 = driver.find_element_by_xpath(kk)
        ActionChains(driver).move_to_element(element1).click(element1).perform()
        #누른다음에 잠깐 쉼
        time.sleep(0.3)          
        time.sleep(0.3)
        k=k+10
    except:
        pass
    


# 다내린다음에 모든것을 파싱하려고함
import pandas as pd

#데이터 프레임으로 틀을 만들어줌
df = pd.DataFrame(columns=['크롤링시간','지역','가게이름','아이디', '몇시간전', '주문내용', '댓글','맛','양','배달','사장', '사진주소'])
#파싱 명령
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

stat = soup.find("span","gps-status ng-binding").text
shop = soup.find("span","restaurant-name ng-binding").text
# 리스트를 찾기위해 스탯과 숍을 앞에다가 걸음
li = soup.find_all("li","list-group-item star-point ng-scope")
# 리스트의 갯수대로 포문을 돌림

for i in range(len(li)):
    li = soup.find_all("li","list-group-item star-point ng-scope")[i]
    # 한개의 리스트 찾기# 아이디 찾기
    id = li.find("span","review-id ng-binding").text
    #몇시간전 찾음
    past = li.find("span","review-time ng-binding").text
    #주문을 찾음
    order = li.find("div","order-items default ng-binding").text.replace("\n","").replace("  ","")
    #이미지가 있을때만 돌아감
    try:
        img = li.find("img").get("src")
    except:
        img = "이미지는 없어요"
    #코멘트를 찾음
    com = li.find("p","ng-binding").text.replace("\n","")
    
    #맛 점수
    mat = li.find_all("span", "points ng-binding")[0].text
    #양 점수
    yang = li.find_all("span", "points ng-binding")[1].text
    #배달점수
    beadal = li.find_all("span", "points ng-binding")[2].text
    #사장리뷰를 새롭게 만듦
    sajang1 = li.find('div','review-answer ng-scope')
    
    try:
        sajang = sajang1.find("p").text
        sajang = re.compile('[^ ㄱ-ㅣ가-힣]+').sub('', sajang )  
      
    except:
        sajang = "----사장님 답글이 없어요 ----"
    
    
    
    #데이터 프레임으로 만든틀에다가 집어 넣음
    df=df.append({'크롤링시간':past,'지역':stat,'가게이름':shop,
                  '아이디':id, '몇시간전':past, '주문내용':order, '댓글':com,
                  '맛': mat,'양': yang,'배달':beadal,'사장':sajang, '사진주소':img}, ignore_index=True)
    

# df 를 만든것을 csv로 만들어줌
df.to_csv("~/Desktop/요기요-{s}-{sh}.csv".format(s=stat, sh = shop), index=False, mode='w', encoding='utf-8-sig')    

#여기까지 시간을 측정함

etime = time.time()
ttime = etime - stime
# 시간을 빼서 앞에것과 몇초가지났는지 확인해봄
print(ttime)
# driver.back() #뒤로가기
df

0  번 긁었어요
2.7598626613616943


,크롤링시간,지역,가게이름,아이디,몇시간전,주문내용,댓글,맛,양,배달,사장,사진주소
0,10시간 전,본오동,치킨플러스-안산본오점,li**님,10시간 전,"순살 바삭담백한후라이드/1(치킨 선택(순살)),떡볶이 대（2~3인용）보통맛/1",매번 시키고 양도 푸짐하구 맛도 좋아요,5,5,5,----사장님 답글이 없어요 ----,이미지는 없어요
1,11시간 전,본오동,치킨플러스-안산본오점,fp**님,11시간 전,（한정판）치플백립（2인이상）/1,오늘도 엄청빨리오셧네요 맛나게먹고잇네요 단점은 갈비가너무뻣뻣해요 다음엔 조리시간을...,5,5,5,----사장님 답글이 없어요 ----,https://rev-static.yogiyo.co.kr/922b1bee327bf4...
2,12시간 전,본오동,치킨플러스-안산본오점,ms**님,12시간 전,순살 반반세트/1(순살 반반 치킨 선택(순살 꿔바로우치킨)),치플은 먹을 때마다 너무 맛있네요!,5,5,5,----사장님 답글이 없어요 ----,https://rev-static.yogiyo.co.kr/53a379985c2b2d...
3,16시간 전,본오동,치킨플러스-안산본오점,rl**님,16시간 전,"순살두마리（100%닭다리살）/1(치킨 선택(순살),순살 두마리 치킨 선택 1(순살눈...",배달도 빠르고 양도 엄청 많았어요 다음에 또 시켜먹을거 같아요~!,5,5,5,소중한 리뷰 감사드립니다우리고객님맛있게 드셨길 바랍니다저희 치플은 가성비와 가심비 ...,이미지는 없어요
4,22시간 전,본오동,치킨플러스-안산본오점,wo**님,22시간 전,호로록닭볶이（2~4인용）/1(음료(콜라 1.25L 변경)),배송이 빨라요 ㅎㅎㅎㅎ,5,5,5,소중한 리뷰 감사드립니다우리고객님맛있게 드셨길 바랍니다저희 치킨플러스는항상 맛있는 ...,https://rev-static.yogiyo.co.kr/089bf03c86a75b...
5,어제,본오동,치킨플러스-안산본오점,ji**님,어제,"극장판치킨（고추치즈편）/1,똥집튀김（소）/1",극장판치킨이 뭔가 궁금했어요!! 똥집튀김이 있는 지 몰랐는데 보다가 있길래 시켜서 ...,5,5,5,소중한 리뷰 감사드립니다우리고객님맛있게 잘드셔주셔서 너무너무 감사드립니다극장판치킨과...,https://rev-static.yogiyo.co.kr/278919a4a81a05...
6,어제,본오동,치킨플러스-안산본오점,do**님,어제,뼈 반반세트/1(반반 치킨 선택(양념치킨)),맛있게 잘먹었습니다 ㅎㅎ,5,5,5,소중한 리뷰 감사드립니다우리고객님맛있게 잘드셔주셔서 너무너무 감사드립니다고객님께서 ...,https://rev-static.yogiyo.co.kr/6485a5eaa27f74...
7,어제,본오동,치킨플러스-안산본오점,ks**님,어제,"순살두마리（100%닭다리살）/1(치킨 선택(더바삭한순살（바삭볼추가）),순살 두마리 ...",자주시켜먹어요 떡복이도맛잇구요.맛나요맛나요,5,4,5,소중한 리뷰 감사드립니다우리고객님맛있게 드셔주셔서 너무너무 감사드립니다역쉬 떡볶이에...,이미지는 없어요
8,어제,본오동,치킨플러스-안산본오점,se**님,어제,순살 강정치킨/1(치킨 선택(더바삭한순살（바삭볼추가）)),사장님 진짜 맛있어요~ 또시켜먹을께요^^ 번창하세요~,5,5,4,소중한 리뷰 감사드립니다우리고객님맛있게 드셔주셔서 너무너무 감사드립니다고객님께서 만...,이미지는 없어요
9,어제,본오동,치킨플러스-안산본오점,05**님,어제,"치킨＋호로록 떡볶이（매운맛）/1(치킨선택(순살 간장치킨),떡볶이 옵션 추가(（떡볶이...",배달이 한시간 넘게걸리네요,5,5,1,소중한 리뷰 감사드립니다우리고객님맛있게 드셨길 바랍니다아공 배달이 늦어 아쉬우셨군요...,이미지는 없어요
